In [1]:
import ddm_data_simulation as ddm_sim
import scipy as scp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
#import kde_class
import multiprocessing as mp
import psutil
import pickle 
import os
import re

import kde_training_utilities as kde_util
import kde_class as kde

In [2]:
kde_util.kde_train_test_from_simulations_flexbound(base_simulation_folder = os.getcwd() + '/data_storage/kde/base_simulations/',
                                         target_folder = os.getcwd() + '/data_storage/kde/kde_ddm_flexbound_train_test_extended_params_2/',
                                         n_total = 55000000,
                                         p_train = 0.9,
                                         mixture_p = [0.8, 0.1, 0.1],
                                         model = 'ddm_flexbound',
                                         print_info = False,
                                         target_file_format = 'pickle',
                                         n_files_max = 'all')

403
806
1209
no_base_data reported
1612
2015
2418
no_base_data reported
2821
3224
3627
4030
4433
4836
5239
5642
6045
[[2.01]]
no_base_data reported
6448
6851
7254
7657
8060
8463
8866
no_base_data reported
9269
9672
10075
10478
10881
[[1.36]]
no_base_data reported
11284
11687
12090
12493
12896
13299
13702
14105
no_base_data reported
14508
14911
15314
15717
16120
16523
16926
no_base_data reported
17329
17732
18135
18538
18941
19344
19747
20150
20553
20956
21359
21762
22165
22568
22971
23374
23777
24180
24583
24986
25389
25792
26195
26598
27001
27404
27807
28210
28613
29016
29419
29822
30225
30628
31031
31434
31837
32240
32643
33046
33449
33852
34255
34658
[[3.12]]
no_base_data reported
35061
no_base_data reported
35464
35867
36270
36673
37076
37479
37882
38285
[[0.32]]
no_base_data reported
38688
no_base_data reported
39091
39494
[[4.33]]
no_base_data reported
39897
40300
no_base_data reported
40703
41106
41509
41912
42315
42718
43121
43524
43927
44330
44733
45136
[[0.71]]
no_base_data r

343759
344162
344565
no_base_data reported
344968
345371
345774
346177
346580
no_base_data reported
346983
347386
347789
348192
348595
348998
349401
349804
350207
350610
351013
351416
no_base_data reported
351819
352222
352625
353028
353431
353834
354237
354640
355043
355446
355849
356252
356655
357058
357461
357864
358267
358670
359073
359476
359879
360282
360685
361088
361491
361894
362297
no_base_data reported
362700
363103
363506
363909
364312
364715
365118
365521
no_base_data reported
365924
no_base_data reported
366327
366730
367133
no_base_data reported
367536
367939
368342
368745
369148
369551
369954
370357
370760
371163
371566
371969
372372
372775
373178
373581
373984
374387
374790
no_base_data reported
375193
375596
375999
376402
376805
no_base_data reported
377208
[[5.01]]
no_base_data reported
377611
no_base_data reported
378014
378417
378820
379223
no_base_data reported
379626
380029
380432
380835
no_base_data reported
381238
381641
no_base_data reported
382044
382447
3828

704847
705250
no_base_data reported
705653
706056
706459
706862
707265
707668
708071
708474
708877
709280
709683
710086
710489
710892
711295
711698
712101
712504
[[2.22]]
no_base_data reported
712907
no_base_data reported
713310
713713
714116
[[2.4]]
no_base_data reported
714519
714922
715325
715728
716131
716534
716937
717340
717743
718146
718549
718952
719355
719758
720161
720564
720967
721370
no_base_data reported
721773
[[1.32]]
no_base_data reported
722176
722579
722982
723385
723788
724191
724594
no_base_data reported
724997
725400
725803
726206
726609
727012
727415
727818
728221
728624
729027
729430
729833
730236
730639
731042
731445
731848
732251
732654
733057
733460
no_base_data reported
733863
734266
734669
735072
735475
735878
736281
736684
737087
no_base_data reported
737490
no_base_data reported
737893
no_base_data reported
738296
738699
739102
no_base_data reported
739505
739908
740311
no_base_data reported
740714
741117
741520
741923
742326
742729
743132
743535
743938
74

1040546
1040949
[[2.39]]
no_base_data reported
1041352
1041755
1042158
1042561
1042964
no_base_data reported
1043367
1043770
1044173
1044576
1044979
1045382
1045785
1046188
no_base_data reported
1046591
1046994
1047397
1047800
1048203
1048606
1049009
1049412
1049815
1050218
1050621
1051024
1051427
1051830
1052233
1052636
1053039
1053442
no_base_data reported
1053845
1054248
1054651
1055054
1055457
1055860
1056263
1056666
1057069
1057472
1057875
1058278
1058681
no_base_data reported
1059084
1059487
[[1.43]]
no_base_data reported
1059890
1060293
1060696
1061099
1061502
1061905
1062308
1062711
1063114
1063517
1063920
1064323
1064726
1065129
1065532
1065935
no_base_data reported
1066338
1066741
1067144
1067547
1067950
1068353
1068756
1069159
1069562
1069965
1070368
1070771
no_base_data reported
1071174
1071577
1071980
1072383
1072786
1073189
no_base_data reported
1073592
1073995
no_base_data reported
1074398
no_base_data reported
1074801
1075204
1075607
1076010
1076413
1076816
no_base_data

1354080
1354483
1354886
1355289
1355692
1356095
1356498
1356901
1357304
1357707
1358110
1358513
1358916
1359319
1359722
1360125
1360528
1360931
1361334
1361737
1362140
1362543
1362946
1363349
1363752
1364155
1364558
1364961
1365364
1365767
1366170
1366573
1366976
1367379
1367782
1368185
1368588
no_base_data reported
1368991
1369394
1369797
1370200
1370603
1371006
1371409
1371812
1372215
1372618
1373021
1373424
1373827
1374230
1374633
1375036
1375439
1375842
1376245
1376648
1377051
1377454
1377857
1378260
1378663
1379066
1379469
1379872
1380275
1380678
1381081
1381484
1381887
1382290
1382693
1383096
1383499
1383902
1384305
1384708
1385111
1385514
1385917
1386320
1386723
1387126
1387529
no_base_data reported
1387932
1388335
1388738
1389141
1389544
1389947
1390350
1390753
1391156
1391559
1391962
[[1.57]]
no_base_data reported
1392365
1392768
1393171
1393574
1393977
1394380
1394783
1395186
1395589
1395992
[[2.3]]
no_base_data reported
1396395
1396798
1397201
1397604
1398007
1398410
1398813

...

53516788
no_base_data reported
53517191
53517594
53517997
53518400
no_base_data reported
53518803
53519206
53519609
53520012
53520415
53520818
53521221
53521624
53522027
53522430
53522833
53523236
53523639
53524042
53524445
53524848
53525251
53525654
53526057
53526460
53526863
53527266
53527669
53528072
53528475
53528878
53529281
53529684
53530087
53530490
53530893
no_base_data reported
53531296
53531699
no_base_data reported
53532102
53532505
53532908
53533311
53533714
53534117
53534520
53534923
53535326
53535729
53536132
53536535
53536938
53537341
53537744
53538147
53538550
53538953
53539356
53539759
53540162
53540565
53540968
53541371
53541774
53542177
53542580
53542983
53543386
53543789
53544192
53544595
53544998
53545401
no_base_data reported
53545804
53546207
53546610
no_base_data reported
53547013
53547416
53547819
53548222
53548625
53549028
53549431
53549834
53550237
53550640
53551043
53551446
53551849
53552252
53552655
53553058
53553461
[[2.38]]
no_base_data reported
53553864


53811381
53811784
53812187
53812590
53812993
53813396
no_base_data reported
53813799
53814202
53814605
53815008
53815411
53815814
no_base_data reported
53816217
53816620
53817023
53817426
53817829
53818232
53818635
53819038
53819441
[[0.64]]
no_base_data reported
53819844
53820247
53820650
53821053
53821456
53821859
53822262
53822665
53823068
53823471
53823874
53824277
53824680
53825083
53825486
53825889
53826292
53826695
53827098
53827501
53827904
53828307
53828710
53829113
53829516
53829919
53830322
53830725
[[1.7]]
no_base_data reported
53831128
53831531
53831934
no_base_data reported
53832337
53832740
53833143
53833546
53833949
53834352
53834755
53835158
53835561
53835964
53836367
53836770
53837173
53837576
53837979
53838382
53838785
53839188
53839591
53839994
53840397
53840800
53841203
53841606
53842009
no_base_data reported
53842412
53842815
53843218
53843621
53844024
no_base_data reported
53844427
53844830
53845233
53845636
no_base_data reported
53846039
53846442
53846845
538472

54101541
54101944
54102347
54102750
54103153
54103556
54103959
54104362
54104765
54105168
[[1.03]]
no_base_data reported
54105571
54105974
54106377
no_base_data reported
54106780
54107183
54107586
no_base_data reported
54107989
54108392
54108795
54109198
54109601
54110004
54110407
54110810
54111213
54111616
54112019
54112422
54112825
54113228
54113631
54114034
54114437
54114840
54115243
54115646
54116049
54116452
54116855
54117258
54117661
54118064
54118467
no_base_data reported
54118870
54119273
54119676
54120079
54120482
54120885
54121288
54121691
54122094
54122497
54122900
54123303
54123706
54124109
54124512
54124915
54125318
54125721
54126124
no_base_data reported
54126527
54126930
54127333
54127736
54128139
54128542
54128945
54129348
54129751
no_base_data reported
54130154
54130557
54130960
no_base_data reported
54131363
no_base_data reported
54131766
54132169
54132572
54132975
54133378
54133781
54134184
54134587
54134990
54135393
54135796
54136199
54136602
no_base_data reported
5

54377596
54377999
[[0.48]]
no_base_data reported
54378402
54378805
54379208
54379611
54380014
54380417
54380820
54381223
54381626
54382029
54382432
no_base_data reported
54382835
54383238
54383641
54384044
54384447
54384850
[[0.47]]
no_base_data reported
54385253
54385656
54386059
54386462
54386865
54387268
54387671
54388074
no_base_data reported
54388477
54388880
54389283
54389686
54390089
54390492
54390895
54391298
54391701
54392104
54392507
54392910
54393313
54393716
54394119
54394522
54394925
54395328
54395731
54396134
54396537
54396940
54397343
54397746
54398149
54398552
54398955
54399358
no_base_data reported
54399761
54400164
54400567
54400970
54401373
54401776
54402179
54402582
54402985
54403388
54403791
54404194
54404597
54405000
54405403
54405806
54406209
54406612
54407015
54407418
54407821
54408224
54408627
54409030
54409433
54409836
54410239
no_base_data reported
54410642
54411045
54411448
54411851
54412254
54412657
54413060
54413463
54413866
54414269
54414672
54415075
5441

54671786
54672189
54672592
54672995
54673398
54673801
54674204
54674607
54675010
54675413
54675816
54676219
54676622
54677025
54677428
54677831
54678234
no_base_data reported
54678637
54679040
54679443
no_base_data reported
54679846
54680249
54680652
54681055
[[2.2]]
no_base_data reported
54681458
54681861
54682264
no_base_data reported
54682667
[[5.68]]
no_base_data reported
54683070
54683473
54683876
no_base_data reported
54684279
no_base_data reported
54684682
54685085
54685488
54685891
54686294
54686697
no_base_data reported
54687100
54687503
54687906
no_base_data reported
54688309
54688712
54689115
54689518
54689921
54690324
54690727
54691130
54691533
54691936
54692339
54692742
54693145
no_base_data reported
54693548
54693951
54694354
54694757
54695160
54695563
54695966
54696369
54696772
54697175
54697578
54697981
54698384
54698787
54699190
54699593
54699996
54700399
54700802
54701205
54701608
54702011
54702414
54702817
54703220
54703623
54704026
54704429
no_base_data reported
547

In [2]:
print(10, 'files read')

10 files read


In [4]:
'10' + str(1)

'101'